# Nichesphere differential co-localization tutorial

Nichesphere is an sc-verse compatible Python library which allows the user to find differentially co-localized cellular niches and biological processes involved in their interactions based on cell type pairs co-localization probabilities in different conditions. Cell type pair co-localization probabilities are obtained in different ways: from deconvoluted Visium 10x / PIC-seq data (probabilities of finding each cell type in each spot / multiplet), or counting cell boundaries overlaps for each cell type pair in single cell spatial data (MERFISH , CODEX ...). This tutorial focuses on defining groups of cells that converge or split in disease (Ischemia) based on differential co-localization. 

Nichesphere also offers the possibility to look at localized differential cell - cell communication based on Ligand-Receptor pairs expression data, such as results from CrossTalkeR [ref]. This is addressed in the localized differential communication tutorial.


## 1. Libraries and functions

In [ ]:
import pandas as pd
import scipy
import seaborn as sns
import glob
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import matplotlib.lines
import networkx as nx
import warnings
import os
import math
import re
import sys
from datetime import datetime
import scanpy as sc
import mudata as md
import numpy as np
from collections import defaultdict
from community_layout.layout_class import CommunityLayout
from cdlib import algorithms, evaluation, NodeClustering
import igraph as ig
import leidenalg as la
import sklearn
warnings.filterwarnings("ignore")

import nichesphere

In [ ]:
# community detection
# download the required python files from https://github.com/lejohnyjohn/signed-louvain
sys.path.insert(1, os.path.join(os.getcwd()+"/signed_louvain")) # change path to your signed louvain folder location
import util
import community_detection as signed_louvain

#!pip install git+https://github.com/alan-turing-institute/SigNet.git
from signet.cluster import Cluster

# cluster assessment
#!pip install ClustAssessPy
from ClustAssessPy import element_sim_matrix, element_consistency

In [ ]:
import random 
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

## 2. Data Import

In [ ]:
adata=sc.read('CODEX_run27-29_anndata_final_rev.h5ad')
adata.obs.head()

## 3. Co-localization

We computed then co-localization probabilities from the cell type probability matrices. Here we got concatenated co-localization sample matrices of cell type x cell type.

Then we reshaped the co-localization data into a matrix of cell type pairs x samples.

(you can have a closer look at these steps in the preprocessing tutorial)

In [ ]:
CTcolocalizationP_ = pd.read_csv('CODEX_run27-29_CTcolocalizationP_rev.csv', index_col=0)

Symmetrize matrices as resulting knn based cell type pair co-localization probabilities are asymmetric

In [ ]:
## run27
CTcolocalizationP = pd.DataFrame()
for smpl in CTcolocalizationP_['sample'].unique():
    t = pd.DataFrame(0, index = CTcolocalizationP_.columns[0:len(CTcolocalizationP_.columns)-1], columns = CTcolocalizationP_.columns[0:len(CTcolocalizationP_.columns)-1])
    for ct1 in CTcolocalizationP_.columns[0:len(CTcolocalizationP_.columns)-1]:
        for ct2 in CTcolocalizationP_.columns[0:len(CTcolocalizationP_.columns)-1]:
            t[ct1][ct2] = np.mean([CTcolocalizationP_[CTcolocalizationP_['sample'] == smpl][ct1][ct2], 
                                 CTcolocalizationP_[CTcolocalizationP_['sample'] == smpl][ct2][ct1]])
    t['sample'] = smpl
    CTcolocalizationP = pd.concat([CTcolocalizationP, t])

CTcolocalizationP.head()

Then we will reshape the co-localization data into a matrix of cell type pairs x samples. \
The sum of the probabilities of every cell type pair in a sample must be = 1.

In [ ]:
colocPerSample = nichesphere.coloc.reshapeColoc(CTcoloc = CTcolocalizationP, complete = 1)
colocPerSample.sum(axis=1)

In [ ]:
colocPerSample.head()

Same cell type interactions will be excluded later on, so we'll have a list of same cell type interaction pairs in order to subset the co-localization table we'll generate in the next step.

In [ ]:
oneCTints = CTcolocalizationP.columns[range(len(CTcolocalizationP.columns)-1)]+'-'+CTcolocalizationP.columns[range(len(CTcolocalizationP.columns)-1)]

Our cell types

In [ ]:
cell_types = CTcolocalizationP.columns[0:len(CTcolocalizationP.columns)-1]
print(cell_types)

**Conditions**

To subset the samples, we will have this dataframe of sample names and conditions. In this case, condition can be inferred from the sample name

In [ ]:
#sampleTypesDF=merfish_obs[['sample', 'status']].drop_duplicates()
sampleTypesDF = adata.obs[['sample', 'condition']].drop_duplicates()
sampleTypesDF.reset_index(drop=True, inplace=True)
sampleTypesDF.columns = ['sample', 'sampleType']
sampleTypesDF.index = sampleTypesDF['sample']

In [ ]:
sampleTypesDF = sampleTypesDF.loc[colocPerSample.index]
sampleTypesDF.head()

In [ ]:
### We have 3 different conditions here: Ctrl=Control , ThPO=Fibrosis model , medium severe , MPL=Fibrosis model , severe
sampleTypesDF.sampleType.unique()

## 4. Differential co-localization analysis

We will test differential co-localization between two different conditions using Wilcoxon rank sums tests

In [ ]:
pvals = [scipy.stats.ranksums(colocPerSample.loc[colocPerSample.index[sampleTypesDF.sampleType=='ThPO'],c], 
                                    colocPerSample.loc[colocPerSample.index[sampleTypesDF.sampleType=='Ctrl'],c]).pvalue for c in colocPerSample.columns]
stat = [scipy.stats.ranksums(colocPerSample.loc[colocPerSample.index[sampleTypesDF.sampleType=='ThPO'],c], 
                                    colocPerSample.loc[colocPerSample.index[sampleTypesDF.sampleType=='Ctrl'],c]).statistic for c in colocPerSample.columns]

DF = pd.DataFrame([colocPerSample.columns, stat, pvals], index=['pairs', 'statistic', 'p-value']).T

DF.index = DF.pairs
DF.head()

Then we will reshape the data to visualize the Wilcoxon test scores in a heatmap and filter non significant co-localization differences using the parameter **p** (in this case, scores with p-values > 0.1 are filtered out)

In [ ]:
HM = nichesphere.tl.pval_filtered_HMdf(testDF=DF, 
                                         oneCTinteractions=oneCTints, 
                                         p=0.1,                             #threshold p-value to filter
                                         cell_types=cell_types)
HM.head() # data type: pd.DataFrame

Now we can plot the differential co-localization scores heatmap

In [ ]:
sns.set(font_scale=1)
plot = sns.clustermap(HM, cmap='vlag', center=0, method='ward', cbar_kws={'label': 'diffColoc. Score'})

**Differential co-localization network**

To build the differential co-localization network, we will get an **adjacency matrix** (adj) based on the **euclidean distances** among the distributions of significant differential co-localization scores for the different cell types

In [ ]:
HMdist=pd.DataFrame(scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(HM)), 
                    columns=HM.columns, index=HM.index)

HMsimm=1-HMdist/HMdist.max().max()
##Cell pairs with not significant differential co-localization get 0
HMsimm[HM==0]=0

In [ ]:
## Plot heatmap
sns.set(font_scale=0.8)
plot=sns.clustermap(HMsimm, cmap='Blues', method='ward', cbar_kws={'label': 'adjacency'})

A **cell group dictionary** should be used here to visualize different cell groups in different colors. As we don't have cell groups yet, we'll have a dictionary of all cells in one group and a list of one color

In [ ]:
niches_dict={'1_': list(HM.index)}
clist=['#4daf4a']

Now we can plot the differential co-localization network using the **colocNW** function from Nichesphere. This function has many parameters that can be tuned: 

**nodeSize** for example, defines how the size of the nodes will be calculated. Options are 'betweeness', 'pagerank' (both network statistics) and None (all nodes have the same size).
**alpha** indicates the transparency of the edges and in goes from 0 (completely transparent) to 1 (opaque)
**fsize** is the size of the figure (x,y)

This function returns the network with the edge weights corresponding to the diff. coloc. scores (positive and negative)

In [ ]:
plt.rcParams['axes.facecolor'] = "None"
g_colocNW = nichesphere.coloc.colocNW(x_diff=HM,            #differential co-localization matrix
                          adj=HMsimm,                   #adjacency matrix
                          cell_group=niches_dict, 
                          clist=clist, 
                          nodeSize='betweeness',        
                          lab_spacing=9,                #space between node and label
                          alpha=0.4,                    #edges transparency
                          fsize=(12,12))                #figure size

Now we'll do community detection using Louvain. First we will get the network from the adjacency matrix as we won't use the signed weights for this

In [ ]:
gCol_unsigned=nx.from_pandas_adjacency(HMsimm, create_using=nx.Graph)

We will use the community-layout library function **CommunityLayout** to show the communities in a layout suited for this. This function is compatible with networkx (Hagberg et. al., 2008) community detection functions, which will be used internally as indicated by the parameters **community_algorithm** and **community_kwargs**

In [ ]:
## Calculate community layout
cl_louvain_unsigned=CommunityLayout(gCol_unsigned,
        community_compression = 0.4,
        layout_algorithm = nx.spring_layout,
        layout_kwargs = {"k":75, "iterations":1000},
        community_algorithm = nx.algorithms.community.louvain_communities,
        community_kwargs = {"resolution":1.2,  'seed':12, 'weight':'weight'})

Test with signed weights in network

We can extract the communities (niches) as follows:

In [ ]:
d_louvain_unsigned = {index: list(value) for index, value in enumerate(cl_louvain_unsigned.communities())}
print(pd.DataFrame.from_dict(d_louvain_unsigned, orient='index').T.to_string(index=False))

And then name them

In [ ]:
niche_names=['WhitePulp', 'EryMacroMesoredP', 'CASinsHSPCgranMono']
niches_dict=dict(zip(niche_names,list(d_louvain_unsigned.values()))) 
print(pd.DataFrame.from_dict(niches_dict, orient='index').T.to_string(index=False))

And assign them colors to color the network nodes according to their niche

In [ ]:
clist=['#95cf92','#369acc', '#de324c']
niche_cols=pd.Series(clist, index=list(niches_dict.keys()))
niches_df=nichesphere.tl.cells_niche_colors(CTs=cell_types, 
                                            niche_colors=niche_cols, 
                                            niche_dict=niches_dict)
niches_df.head()

Then we can get the node positions to input them to the nichesphere **colocNW** function through the **pos** parameter

In [ ]:
pos=cl_louvain_unsigned.full_positions

Now we plot the niches on the community layout

In [ ]:
plt.rcParams['axes.facecolor'] = "None"

gCol=nichesphere.coloc.colocNW(x_diff=HM, 
                               adj=HMsimm,
                               cell_group=niches_dict, 
                               clist=clist, 
                               nodeSize='betweeness', 
                               layout=None,                         #layout needs to be set to None if we provide node positions
                               lab_spacing=0.05, 
                               thr=1, 
                               alpha=0.4, 
                               fsize=(10,10), 
                               pos=pos,                             #node positions (from the CommunityLayout function)
                               edge_scale=1,                        #edge width
                               legend_ax=[0.7, 0.05, 0.15, 0.2])    #legend position
#Legend
legend_elements1=[plt.Line2D([0], [0], marker="o" ,color='w', markerfacecolor=clist[i], lw=4, 
                             label=list(niches_dict.keys())[i], ms=10) for i in range(len(list(niches_dict.keys())))]
plt.gca().add_artist(plt.legend(handles=legend_elements1,loc='lower left', fontsize=13, title='Niches', 
                                alignment='left'))
#plt.savefig('diffColocNW_CD.pdf')

#### Different Community Detection approaches

In [ ]:
import community_clustering as com_clustering
import imp

imp.reload(com_clustering)

In [ ]:
# DataFrame Table for community detection results
com_table = pd.DataFrame(columns=[
    "method",
    "result_df",
    "vc_map",
    "n_communities",
    "timestamp"
])

In [ ]:
# Choose one of the available methods. The name of the method can also be entered directly as a string
method_name = com_clustering._choose_method(6)
print(f"Choosing community detection method {method_name}")

default_params = com_clustering._get_default_params(method_name)
print(f"Default method parameters: {default_params}.")

In [ ]:
# Method parameters that should be changed in comparison to 
params = {
    "seed": 12,
    "n_clusters": 4
}

In [ ]:
communities_df, vc_map, method_params = com_clustering.community_detection(
    gCol=g_colocNW,
    HM=HM,
    HMsimm=HMsimm,
    method_name=method_name,
    cell_types=cell_types,
    params=params,
    plot=True,
    save_plot=False,
)

row = method_params.copy()
row.update({
    "method": method_name,
    "result_df": communities_df,
    "vc_map": vc_map,
    "n_communities": len(set(vc_map.values())),
    "timestamp": datetime.now(),
})

com_table = pd.concat([com_table, pd.DataFrame([row])], ignore_index=True)


**Comparison signed clustering methods**

In [ ]:
import clustering_metrics as clm

In [ ]:
for idx, row in com_table.iterrows():
    method = row["method"]
    vc_map = row["vc_map"]
    
    g = gCol_unsigned if method in com_clustering._get_unsigned_methods() else g_colocNW

    metrics = clm.evaluate_signed_clustering(g, vc_map)
    
    for k, v in metrics.items():
        com_table.loc[idx, k] = v


In [ ]:
com_table

**Map niches to chells/spots**

## Visualize niches in slices

In [ ]:
com_method = 'spinglass_weights'

In [ ]:
niches_df = com_res[com_method][0]['result_df']

In [ ]:
niche_names = ['Community 1', 'Community 2', 'Community 3', 'Community 4']

In [ ]:
cmap = plt.get_cmap("rainbow")   
colourlist = [mcolors.to_hex(cmap(i)) for i in np.linspace(0, 1, len(niche_names))]
clist = ['#95cf92','#369acc', '#de324c', '#ffff00']

cols=pd.Series(clist, index=niche_names)
cols

In [ ]:
all_niches=pd.DataFrame()
for smpl in  list(sampleTypesDF['sample'].unique()):
    tmp=adata[adata.obs['sample']==smpl].copy()
    tmp.obs['niche']='0_'
    for ct in niches_df.cell:
        tmp.obs.niche[tmp.obs.clusters_named==ct]=niches_df.niche[niches_df.cell==ct][0]
    tmp.obs.niche=tmp.obs.niche.astype('category')
    for c in np.setdiff1d(niches_df.niche.cat.categories,tmp.obs.niche.cat.categories):
        tmp.obs.niche = tmp.obs.niche.cat.add_categories(c)
    tmp.obs.niche=tmp.obs.niche.cat.reorder_categories(niches_df.niche.cat.categories)
    tmp.uns['niche_colors']=cols[niches_df.niche.cat.categories]
    
    all_niches=pd.concat([all_niches, tmp.obs[['niche', 'sample', 'condition']]])

In [ ]:
all_niches

In [ ]:
adata.obs['ThPO_Ctrl_niche']='0_'
for ct in niches_df.cell:
    adata.obs.ThPO_Ctrl_niche[adata.obs.clusters_named==ct]=niches_df.niche[niches_df.cell==ct][0]
adata.obs.ThPO_Ctrl_niche=adata.obs.ThPO_Ctrl_niche.astype('category')
adata.obs.ThPO_Ctrl_niche=adata.obs.ThPO_Ctrl_niche.cat.reorder_categories(niches_df.niche.cat.categories)
adata.uns['ThPO_Ctrl_niche_colors']=list(cols[niches_df.niche.cat.categories])

In [ ]:
fig, axes = plt.subplots(3, 5, figsize=(25, 16))
plt.close(fig)
for idu,smpl in enumerate(list(adata.obs['sample'][adata.obs.condition=='Ctrl'].unique())):  
    ad=adata[adata.obs['sample']==smpl].copy()
    sc.pl.spatial(ad, color=['ThPO_Ctrl_niche'], spot_size = 20, use_raw=False, title=smpl, ax=axes[0][idu])

In [ ]:
for idu,smpl in enumerate(list(adata.obs['sample'][adata.obs.condition=='ThPO'].unique())):  
    ad=adata[adata.obs['sample']==smpl].copy()
    sc.pl.spatial(ad, color=['ThPO_Ctrl_niche'], spot_size = 20, use_raw=False, title=smpl, ax=axes[1][idu])

In [ ]:
for idu,smpl in enumerate(list(adata.obs['sample'][adata.obs.condition=='MPL'].unique())):  
    ad=adata[adata.obs['sample']==smpl].copy()
    sc.pl.spatial(ad, color=['ThPO_Ctrl_niche'], spot_size = 20, use_raw=False, title=smpl, ax=axes[2][idu])

In [ ]:
fig.tight_layout()
fig

In [ ]:
fig.savefig(os.path.join("figures",'vis_niche_slices_'+com_method+'.png'), dpi=900)

In [ ]:
#adata.write_h5ad('./CODEX_run27-29_anndata_final_ThPO_Ctrl_niche_rev.h5ad')

In [ ]:
clusts_in_smpls=adata.obs.groupby(['condition', 'ThPO_Ctrl_niche']).size().unstack()
clust_cols=cols[clusts_in_smpls.columns]
clusts_in_smpls.plot.bar(stacked=True, color=clust_cols)

In [ ]:
clusts_in_smpls_norm=clusts_in_smpls.T/clusts_in_smpls.T.sum()
clusts_in_smpls_norm.T.plot.bar(stacked=True, color=clust_cols)